In [1]:
%load_ext autoreload
%autoreload 2

%pwd
%matplotlib inline

import pandas as pd
from tqdm.notebook import tqdm

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

In [2]:
import pathlib, sys
import logging

DATA_FOLDER = pathlib.Path("/nfs/data/students/bsparks/mdti4py-dataset-pool/cdt4py/")
assert DATA_FOLDER.is_dir()

logger = logging.getLogger(name=__name__)
logger.handlers.clear()
logger.setLevel(level=logging.DEBUG)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(fmt=logging.Formatter('%(asctime)s | %(levelname)s : %(message)s'))
logger.addHandler(handler)

logger.info("Hello World")

2023-09-18 13:47:57,643 | INFO : Hello World


In [3]:
from typet5.data import GitRepo
from scripts.infer.structure import CrossDomainTypes4Py

dataset = CrossDomainTypes4Py(dataset_root=DATA_FOLDER)
logger.debug(dataset)



from scripts.infer.structure import AuthorRepo
from typet5.data import GitRepo


class CDT4PyRepo(GitRepo):
    def __init__(self, author_repo: AuthorRepo) -> None:
        super().__init__(
            author=author_repo.author,
            name=author_repo.repo,
            url=None,
            stars=-1,
            forks=-1
        )

class CDT4PyFlaskRepo(CDT4PyRepo):
    def repo_dir(self, repos_dir: pathlib.Path) -> pathlib.Path:
        return repos_dir / "flask" / self.authorname()

    def authorname(self) -> str:
        assert self.name.startswith("flask-"), f"Unexpected author: {self.name}"
        return f"{self.author}/{self.name[6:]}"
    
class CDT4PyNumpyRepo(CDT4PyRepo):
    def repo_dir(self, repos_dir: pathlib.Path) -> pathlib.Path:
        return repos_dir / "numpy" / self.authorname()

    def authorname(self) -> str:
        assert self.name.startswith("numpy-"), f"Unexpected author: {self.name}"
        return f"{self.author}/{self.name[6:]}"

2023-09-18 13:48:02.423329: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 13:48:03.334403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-09-18 13:48:07,255 | DEBUG : CrossDomainTypes4Py @ /nfs/data/students/bsparks/mdti4py-dataset-pool/cdt4py


In [4]:
cdt4py_author_repos: dict[AuthorRepo, pathlib.Path] = {
    dataset.author_repo(repository): repository
    for repository in dataset.project_iter()
}
display(len(cdt4py_author_repos))

8140

# Remove BetterTypes4Py, Typilus and Type4Py datasets from CrossDomainTypes4Py

In [5]:
#!pip install cd4py
#!pip install "typing_extensions==4.5.0"

Number of source code files: 2,604,611                                                                                
[I 2023-09-07 14:03:50.700 ServerApp] Saving file at /experiments/00datasets/cdt4py.ipynb                            │Total number of tokens: 1,268,703,455                                                                                 
[I 2023-09-07 14:05:50.926 ServerApp] Saving file at /experiments/00datasets/cdt4py.ipynb                            │100%|█████████████████████████████████████████████████████████████████████| 2604611/2604611 [05:56<00:00, 7315.22it/s]
[W 2023-09-07 17:52:42.791 ServerApp] WebSocket ping timeout after 119980 ms.                                        │100%|█████████████████████████████████████████████████████████████████████| 2604611/2604611 [12:04<00:00, 3596.45it/s]
[I 2023-09-07 17:52:47.792 ServerApp] Starting buffering for 86de0c3c-afd7-4dac-bf0d-24991e3d37cb:2196c57f-60da-4220-│***********************Vectorize pre-processed source code files using TF-IDF**********************                   
8563-671aa53816b7                                                                                                    │100%|█████████████████████████████████████████████████████████████████████| 2604611/2604611 [21:26<00:00, 2023.87it/s]
[I 2023-09-08 17:42:51.424 ServerApp] 302 GET / (@127.0.0.1) 0.92ms                                                  │**************************Building KNN index and finding nearest neighbors*************************                   
[W 2023-09-08 17:42:53.855 LabApp] Could not determine jupyterlab build status without nodejs                        │100%|█████████████████████████████████████████████████████████████████████| 2604611/2604611 [05:55<00:00, 7336.19it/s]
[I 2023-09-08 17:42:54.006 ServerApp] Connecting to kernel cf8cb073-e5b3-4c1c-b385-c6ccc37efb55.                     │100%|██████████████████████████████████████████████████████████████████████| 2604611/2604611 [50:09<00:00, 865.38it/s]
[I 2023-09-08 17:42:54.058 ServerApp] Connecting to kernel 9fc37da9-cbb6-4566-928d-99f2fee6a601.                     │*******************************Finding exact and near duplicate files******************************                   
[I 2023-09-08 17:42:54.097 ServerApp] Connecting to kernel 86de0c3c-afd7-4dac-bf0d-24991e3d37cb.                     │100%|█████████████████████████████████████████████████████████████████████| 2604611/2604611 [07:46<00:00, 5580.31it/s]
[I 2023-09-08 17:42:54.332 ServerApp] Starting buffering for cf8cb073-e5b3-4c1c-b385-c6ccc37efb55:fc69da8c-75c8-4c77-│*********************Report duplication stats & saving detected duplicate files********************                   
9f70-ba2e4aa42f86                                                                                                    │Number of duplicated files: 2,417,250 (92.81%)                                                                        
[I 2023-09-08 17:42:54.333 ServerApp] Starting buffering for 9fc37da9-cbb6-4566-928d-99f2fee6a601:c7f7880b-5671-4bba-│Number of detected clusters: 184,937                                                                                  
be2d-b039572025d3                                                                                                    │Avg. number of files per clones: 13.07                                                                                
[I 2023-09-08 17:42:54.334 ServerApp] Starting buffering for 86de0c3c-afd7-4dac-bf0d-24991e3d37cb:3b942345-73e7-49ea-│Median number of files per clones: 4.00                                                                               
81d8-45e8e8e83e92                                                                                                    │Duplication ratio: 85.71%                                                                                             

In [6]:
!cd4py --help

usage: cd4py [-h] --p P --od OD --ot OT [--d D] [--th TH] [--k K] [--tr TR]

Code De-Duplication for Python

options:
  -h, --help  show this help message and exit
  --p P       Path to Python projects
  --od OD     Output folder to store detected duplicate files.
  --ot OT     Output folder to store tokenized files.
  --d D       Dimension of TF-IDF vectors [default: 2048].
  --th TH     Threshold to identify duplicate files [default: 0.95].
  --k K       Number of nearest neighbor [default: 10].
  --tr TR     Number trees to build the index. More trees gives higher
              precision but slower [default: 20].


In [7]:
%%time
#!cd4py --p /nfs/data/students/bsparks/mdti4py-dataset-pool --ot /nfs/data/students/bsparks/mdti4py-dataset-pool-toks --od /nfs/data/students/bsparks/mdti4py-datasets-dedup

# Run in bash shell

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.06 µs


In [8]:
from dpu_utils.utils.dataloading import load_jsonl_gz
from libcst import codemod

import json, random, collections
import pprint

# Get size of each project
projects_by_size = collections.Counter({
    project: len(codemod.gather_files([project]))
    for project in tqdm(dataset.project_iter(), total=len(cdt4py_author_repos))
})

  0%|          | 0/8140 [00:00<?, ?it/s]

In [9]:
df = pd.DataFrame.from_dict({
    "name": [project.name for project in tqdm(dataset.project_iter(), total=len(cdt4py_author_repos))]
})

display(df.value_counts().head())

  0%|          | 0/8140 [00:00<?, ?it/s]

name              
home-assistant        135
incubator-superset     61
Python                 59
pandas                 45
allennlp               43
Name: count, dtype: int64

In [10]:
# Collect only cdt4py files
cdt4py_clusters = list[dict]()
cdt4py_path = pathlib.Path("/nfs/data/students/bsparks/mdti4py-dataset-pool/cdt4py")
for cluster in load_jsonl_gz("/nfs/data/students/bsparks/mdti4py-datasets-dedup/duplicates.jsonl.gz"):
    no_sites = list()
    for file in cluster:
        if all(forbidden not in file.lower() for forbidden in ["venv", "site-packages", "lib", "scripts"]):
            no_sites.append(file)
    
    pathed = map(pathlib.Path, no_sites)
    # print(pathed)
    cdt4py_files = list(filter(lambda p: p.is_relative_to(cdt4py_path), pathed))
    if not cdt4py_files:
        continue

    segmented_files = collections.defaultdict[tuple, list](list)
    for fpath in cdt4py_files:
        from_dataset = fpath.relative_to(cdt4py_path)
        category, author, user, *_ = from_dataset.parts
        key = (category, author, user)
        segmented_files[key].append(fpath)
    
    cdt4py_clusters.append(segmented_files)

In [11]:
#print(next(iter(projects_by_size)))
cdt4py_clusters_wip = cdt4py_clusters[:]
cdt4py_dedupped = list[pathlib.Path]()

vendored = {
    ("flask", "kichappa", "QC"),
    ("numpy", "kichappa", "QC"),
}

for project, _ in tqdm(projects_by_size.most_common(), total=len(projects_by_size)):
    from_dataset = project.relative_to(cdt4py_path)
    category, author, user, *_ = from_dataset.parts
    key = (category, author, user)
    if key in vendored:
        continue

    redundant = set[int]()
    for i, cdt4py_cluster in enumerate(cdt4py_clusters_wip):
        if cluster_files := cdt4py_cluster.get(key):
            redundant.add(i)
            cdt4py_dedupped.append(random.choice(cluster_files).relative_to(cdt4py_path))

    if redundant:
        cdt4py_clusters_wip = [cluster for i, cluster in enumerate(cdt4py_clusters_wip) if i not in redundant]

  0%|          | 0/8140 [00:00<?, ?it/s]

In [96]:
import os
cdt4py_dedupped = list(map(str, cdt4py_dedupped))
print(cdt4py_dedupped[:3])

files_by_segment = (
    pd.Series(cdt4py_dedupped, name="file").str \
    .split(pat=os.sep, n=3, expand=True) \
    .rename(columns=dict(enumerate(["category", "user", "repository", "file"])))
)
files = pd.concat([
    files_by_segment[["category", "user", "repository"]].apply(os.sep.join, axis=1).rename("prefix"),
    files_by_segment["file"]
], axis="columns")
    
display(files.head(n=20), files.shape)

['numpy/rayhaneHamoumi/arduino/python/TemperatureSensor_Python.py', 'numpy/Peipeixuan/muxing-crowdfunding/mainapp/migrations/0003_auto_20200829_1246.py', 'numpy/Peipeixuan/muxing-crowdfunding/proj02/settings.py']


,prefix,file
0,numpy/rayhaneHamoumi/arduino,python/TemperatureSensor_Python.py
1,numpy/Peipeixuan/muxing-crowdfunding,mainapp/migrations/0003_auto_20200829_1246.py
2,numpy/Peipeixuan/muxing-crowdfunding,proj02/settings.py
3,numpy/Peipeixuan/muxing-crowdfunding,mainapp/migrations/0005_auto_20200829_1456.py
4,numpy/Peipeixuan/muxing-crowdfunding,mainapp/migrations/0004_auto_20200829_1442.py
5,numpy/Peipeixuan/muxing-crowdfunding,manage.py
6,numpy/Peipeixuan/muxing-crowdfunding,mainapp/migrations/0002_auto_20200824_1438.py
7,flask/danfordshadrack/ulanga-dash,script.py
8,flask/danfordshadrack/ulanga-dash,main.py
9,flask/jovanglig/Disaster_Response_App,app/run.py


(117145, 2)

In [13]:
print("projects:", files["prefix"].nunique())
print("largest projects:", files["prefix"].value_counts().head())
print("file count:", files["prefix"].value_counts().sum())

projects: 3566
largest projects: prefix
numpy/sviete/AIS-home-assistant    4661
flask/kfserving/kfserving          2797
flask/brycepg/pylint-corpus        2571
flask/gyhd/python_study            2277
numpy/Pandinosaurus/airflow        1746
Name: count, dtype: int64
file count: 117145


In [ ]:
display(files["prefix"].value_counts())

In [15]:
new_repos = []

for repository in tqdm(files["prefix"].unique()):
    test_repo_path = cdt4py_path / repository
    if repository.startswith("numpy/"):
        repo = CDT4PyNumpyRepo(dataset.author_repo(test_repo_path))
    elif repository.startswith("flask/"):
        repo = CDT4PyFlaskRepo(dataset.author_repo(test_repo_path))
    else:
        assert False, f"Unknown prefix: {repository}" 
    d = repo.repo_dir(cdt4py_path)
    if d.is_dir():
        new_repos.append(repo)

    # for r in tqdm.tqdm(downloaded_repos, desc=str(cdt4py_repo)):
    #    r.read_last_update(DATA_FOLDER)
logger.info(f"Selected {len(new_repos)} projects")

  0%|          | 0/3566 [00:00<?, ?it/s]

2023-09-18 13:56:00,835 | INFO : Selected 3566 projects


In [16]:
loc_limit = 50000

acceptable_repos = []
all_repos = []
for rep in tqdm(new_repos):
    try:
        loc = rep.count_lines_of_code(DATA_FOLDER)
        if loc < loc_limit:
            acceptable_repos.append(rep)
    except UnicodeDecodeError:
        # nothing we can do
        logger.warning(f"{rep.authorname()} does not pass due to encoding error")
    except Exception as e:
        logger.warning(f"{rep.authorname()} does not pass", exc_info=True)

    else:
        all_repos.append(rep)

  0%|          | 0/3566 [00:00<?, ?it/s]

2023-09-18 13:56:02,990 | WARNING : rayhaneHamoumi/arduino does not pass due to encoding error
2023-09-18 13:56:04,700 | WARNING : Peipeixuan/muxing-crowdfunding does not pass due to encoding error
2023-09-18 13:56:11,209 | WARNING : jovanglig/Disaster_Response_App does not pass due to encoding error
2023-09-18 13:56:14,042 | WARNING : Beomsudev/Git does not pass due to encoding error
2023-09-18 13:56:15,247 | WARNING : garikapatisravani/PythonDeepLearning does not pass due to encoding error
2023-09-18 13:56:17,074 | WARNING : dasanchez11/dog-breed does not pass due to encoding error
2023-09-18 13:56:18,684 | WARNING : cryptowhaler/ISC-POC does not pass due to encoding error
2023-09-18 13:56:21,973 | WARNING : Anubha-Singh/Disease-Mortality-Prediction does not pass due to encoding error
2023-09-18 13:56:22,749 | WARNING : iron-condor/MLSR-LTC does not pass due to encoding error
2023-09-18 13:56:24,720 | WARNING : autist47/MusicToNotes does not pass due to encoding error
2023-09-18 13:5

In [17]:
df = pd.DataFrame.from_dict({
    "name": [repo.name[6:] for repo in tqdm(acceptable_repos, total=len(acceptable_repos))]
})

display(df.value_counts().head())

  0%|          | 0/2907 [00:00<?, ?it/s]

name              
Python                13
openpilot             11
allennlp               9
incubator-superset     8
PySyft                 5
Name: count, dtype: int64

In [18]:
print(
    f"{len(acceptable_repos)}/{len(all_repos)} repos pass readability and LOC checks."
)

2907/3473 repos pass readability checks.


In [30]:
%%capture
def count_repo_annots(rep: CDT4PyRepo) -> tuple[CDT4PyRepo, dict] | None:
    try:
        annotations = rep.collect_annotations(DATA_FOLDER, silent=True)
        if rep.n_type_annots / rep.lines_of_code >= 0.02:
            return rep, annotations
    except Exception as e:
        #logger.warning(f"Failed to count annotations for {rep.name}. Exception: {e}")
        return None

from concurrent.futures import ProcessPoolExecutor, as_completed

with ProcessPoolExecutor(max_workers=64) as executor:
    fs = [executor.submit(count_repo_annots, r) for r in acceptable_repos]
    repo2annotations = [f.result() for f in tqdm(as_completed(fs), total=len(fs))]

In [31]:
import operator

repo2annotations: list[tuple[CDT4PyRepo, dict]] = [r for r in repo2annotations if r is not None]
useful_repos: list[CDT4PyRepo] = list(map(operator.itemgetter(0), repo2annotations))

logger.info(
    f"{len(useful_repos)}/{len(acceptable_repos)} repos are parsable, have enough portions of type annotations"
)

2023-09-19 11:38:23,927 | INFO : 1553/2907 repos are parsable, have enough portions of type annotations


In [50]:
useful_repo_names = []
for repo in useful_repos:
    useful_repo_names.append(f"{repo.name[:5]}/{repo.author}/{repo.name[6:]}")
print(useful_repo_names[:3])

import numpy as np
assert (s := files["prefix"].value_counts().index.isin(useful_repo_names).sum()) == len(useful_repo_names), f"{s} != {len(useful_repo_names)}"

cdt4py_repos = files[files["prefix"].isin(useful_repo_names)]
display(len(cdt4py_repos))
display(cdt4py_repos["prefix"].nunique())

['numpy/Tony-Starkus/udemy-course-guppe', 'flask/uniwue-it4all/it4all-resources', 'numpy/nickderobertis/py-ex-latex']


26957

1553

In [75]:
import random

sized_groups = [group for _, group in cdt4py_repos.groupby("prefix")]
random.Random(42).shuffle(groups)

cdt4py_shuffled_repos = pd.concat(groups).reset_index(drop=True)

In [97]:
weighted = cdt4py_shuffled_repos.groupby("prefix", sort=False)\
  .agg({"file": "count"})
weighted["file"] = (weighted["file"] / weighted["file"].sum()).cumsum()
#display(weighted["file"].head())

validation_repositories, test_repositories = weighted[weighted["file"] < 0.8], weighted[weighted["file"] >= 0.8]
# display(validation_repositories, test_repositories)

validation_split = cdt4py_shuffled_repos[cdt4py_shuffled_repos["prefix"].isin(validation_repositories.index)]
test_split = cdt4py_shuffled_repos[cdt4py_shuffled_repos["prefix"].isin(test_repositories.index)]

display(cdt4py_shuffled_repos.head(), total_files.shape[0])
display(validation_split.head(), validation_split.shape[0])
display(test_split.head(), test_split.shape[0])
display(validation_split.shape[0] + test_split.shape[0])

,prefix,file
0,numpy/marshall-lab/TITAN,tests/features/prep_test.py
1,numpy/JanAlexanderZak/leetspeak,tests/script.py
2,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/exception_utils.py
3,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/factory.py
4,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/train.py


26957

,prefix,file
0,numpy/marshall-lab/TITAN,tests/features/prep_test.py
1,numpy/JanAlexanderZak/leetspeak,tests/script.py
2,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/exception_utils.py
3,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/factory.py
4,numpy/BMW-InnovationLab/BMW-YOLOv4-Training-Au...,src/train.py


21564

,prefix,file
21564,numpy/taikiinoue45/STAD,stad/utils/show_test_results.py
21565,numpy/taikiinoue45/STAD,streamboard/utils/show_test_anomaly_results.py
21566,numpy/taikiinoue45/STAD,stad/datasets/mvtec.py
21567,flask/VWS-Python/vws-python-mock,tests/mock_vws/fixtures/vuforia_backends.py
21568,flask/VWS-Python/vws-python-mock,tests/mock_vws/test_target_list.py


5393

26957

In [114]:
for name, split in zip(("deduplicated", "validation", "test"), (cdt4py_shuffled_repos, validation_split, test_split)):
    print("===", name, "===")
    print("project count:", split["prefix"].nunique())
    print("file count:", split["file"].count())
    print("file count stddev", split.groupby("prefix").agg({"file": "count"})["file"].std())
    
    print("\n")

=== deduplicated ===
project count: 1553
file count: 26957
file count stddev 30.7809954284995


=== validation ===
project count: 1256
file count: 21564
file count stddev 29.72779852642132


=== test ===
project count: 297
file count: 5393
file count stddev 34.92954115892228




In [110]:
with (pathlib.Path("/nfs/data/students/bsparks/mdti4py-datasets-dedup") / "deduplicated.csv").open("w") as f:
    cdt4py_shuffled_repos.to_csv(f, header=["project", "file"], index=False)

with (pathlib.Path("/nfs/data/students/bsparks/mdti4py-datasets-dedup") / "validation.csv").open("w") as f:
    validation_split.to_csv(f, header=["project", "file"], index=False)

with (pathlib.Path("/nfs/data/students/bsparks/mdti4py-datasets-dedup") / "test.csv").open("w") as f:
    test_split.to_csv(f, header=["project", "file"], index=False)